# 图像转换

Opencv主要提供了两个转换函数 warpAffine 和 warpPerspective完成全部的转换的工作


## 仿射变换（Affine transformation）
在讨论仿射变换之前，我们先了解一下欧几里得变换（Euclidean transformation）。欧几里得变换是一种几何变换，任何对象在进行欧几里得变换后会保持原来的形状和大小，严格一些说原空间的任何两个点，在经过转换后，它们的距离不变。欧几里得变换包括旋转，位移，反射以及它们的组合。如何在欧几里得变换上附加偏手性（handedness），反射就不是欧几里得变换了，因为右手在转换后会变成左手。

仿射变换是欧几里得变换的扩展，原空间中的点，直线和面在转换后仍然得以保持，同时它还保持线的并行性。但是其不保持相交线的角度和两个点的距离。仿射变换保持位移（translation），缩放（scaling），反射（reflection），旋转（rotation），错切（shear）等。


### 位移（translation）

移动使用如下的转换矩阵

$ T = \begin{bmatrix}
        1 & 0 & t_x \\
        0 & 1 & t_y 
        \end{bmatrix} $

In [ ]:
import cv2
import numpy as np

读入Image文件

In [ ]:
img = cv2.imread('/Users/davidyon/Desktop/IMG_9261.JPG')

获取Image文件的维度信息

In [ ]:
num_rows,num_cols = img.shape[:2]

设置转换矩阵，向右移动350（tx），向下移动550（ty），对应的矩阵如下：

In [ ]:
move_matrix = np.float32([ [1,0,350], [0,1,550] ])

使用函数 cv2.warpAffine 进行转换，这个函数使用 2x3 的转换矩阵作为输入。另外一个转换函数 cv.warpPerspective 使用 3x3 的转换矩阵作为输入

In [ ]:
img_move = cv2.warpAffine(img, move_matrix, (num_cols+700,num_rows+1100), cv2.INTER_LINEAR, cv2.BORDER_WRAP, 2)

其中，(num_cols+700,num_rows+1100) 是转换后的图像维度，转换的图像宽度增加700，高度增加1100，这样可以使移动（350，550）后的图像显示在正中间。后面两个参数用于指定如何填充移动的边框。

In [ ]:
cv2.imshow('Translation_Move', img_move)
cv2.waitKey()
cv2.destroyAllWindows()
for i in range(1,5):
    cv2.waitKey(1)

之所以后面有循环，是因为在MacOS上destroyAllWindows不起作用，但是增加几个waitKey调用就可以删除窗口

###  旋转 （rotate）
图像旋转使用如下的转化矩阵

$
R=\begin{bmatrix}
    \cos \theta  & -\sin \theta \\
    \sin \theta  & \cos \theta
  \end{bmatrix}
$

其中 $ \theta $ 是逆时针旋转的角度。OpenCV 的函数 `getRotationMatrix2D` 提供针对旋转矩阵的细粒度的控制。我们可以指定图像基于那个点进行旋转，旋转的角度，以及对图像的缩放等参数。一旦我们定义了这个旋转矩阵，我们可以使用 `warpAffine` 函数将这个矩阵应用到任何图像。

In [ ]:
import cv2
import numpy as np
img = cv2.imread('/Users/davidyon/Desktop/IMG_9261.JPG')
num_rows, num_cols = img.shape[:2]
translation_matrix = np.float32([ [1,0,int(0.5*num_cols)], [0,1,int(0.5*num_rows)] ])
rotation_matrix = cv2.getRotationMatrix2D((num_cols, num_rows),30,1)
img_translation = cv2.warpAffine(img, translation_matrix, (2*num_cols, 2*num_rows))
img_rotation = cv2.warpAffine(img_translation,rotation_matrix,(2*num_cols, 2*num_rows))
cv2.imshow('Rotation',img_rotation)
cv2.waitKey()
cv2.destroyAllWindows()
cv2.waitKey(1)

其中，下面的代码是为了避免旋转后有些部分被截掉而增加显示的空间，并将图像向中间移动. 

```python
translation_matrix = np.float32([ [1,0,int(0.5*num_cols)], [0,1,int(0.5*num_rows)] ])
...
img_translation = cv2.warpAffine(img, translation_matrix, (2*num_cols, 2*num_rows))
```

### 缩放（scale）
OpenCV 提供 `resize` 函数对图像进行缩放，并提供相应的缩放参数

In [ ]:
import cv2
img = cv2.imread('/Users/davidyon/Desktop/IMG_9261.JPG')
img_scaled = cv2.resize(img,None,fx=1.2, fy=1.2, interpolation =
                        cv2.INTER_LINEAR)
cv2.imshow('Scaling - Linear Interpolation', img_scaled)
img_scaled = cv2.resize(img,None,fx=1.2, fy=1.2, interpolation =
   cv2.INTER_CUBIC)
cv2.imshow('Scaling - Cubic Interpolation', img_scaled)
img_scaled = cv2.resize(img,(450, 400), interpolation = cv2.INTER_AREA)
cv2.imshow('Scaling - Skewed Size', img_scaled)
cv2.waitKey(5000)
cv2.destroyAllWindows()
cv2.waitKey(1)

### 错切（shear）
要构造一个通用的仿射变换矩阵，我们需要定义控制点。确定了这些控制点后，我们需要确定要把这些控制点映射到何处。对于仿射变换，我们需要做的是在原来的图像中找出三个点，然后确定这三个点在目标图像中的三个位置。下面，我们通过定义原图像中的三个顶点和其相应的目标图像中的三个映射点完成一个错切变换。

In [ ]:
import cv2
import numpy as np
img = cv2.imread("/Users/davidyon/Desktop/IMG_9261.JPG")
rows, cols = img.shape[:2]
src_points = np.float32([[0,0], [cols-1,0], [0,rows-1]])
dst_points = np.float32([[0,0], [int(0.6*(cols-1)),0],
[int(0.4*(cols-1)),rows-1]])
affine_matrix = cv2.getAffineTransform(src_points, dst_points)
img_output = cv2.warpAffine(img, affine_matrix, (cols,rows))
print(img_output.shape)
cv2.imshow('Input', img)
cv2.imshow('Output', img_output)
cv2.waitKey()
cv2.destroyAllWindows()
cv2.waitKey(1)

### 反射（reflect）
同样我们可以通过这种方式完成反射变换。与上面的代码相比，唯一的不同点是定义的三个目标点不同


In [ ]:
import cv2
import numpy as np
img = cv2.imread("/Users/davidyon/Desktop/IMG_9261.JPG")
rows, cols = img.shape[:2]
src_points = np.float32([[0,0], [cols-1,0], [0,rows-1]])
dst_points = np.float32([[cols-1,0], [0,0], [cols-1,rows-1]])
affine_matrix = cv2.getAffineTransform(src_points, dst_points)
img_output = cv2.warpAffine(img, affine_matrix, (cols,rows))
print(img_output.shape)
cv2.imshow('Input', img)
cv2.imshow('Output', img_output)
cv2.waitKey()
cv2.destroyAllWindows()
cv2.waitKey(1)

## 射影变换（Projective transformation）

射影变换也称为单应性（Homography），射影性（projectivity）或者直射（projective collineation）。射影变换是从实射影平面到射影平面的可逆转换，直线在该变换下仍应射为直线。它描述了当观察者视角改变时，被观察物体的感知位置会发生何种变化。射影变换并不保持大小和角度，但会保持重合关系（incidence）和交比（cross-ratio）。
对于射影变换，我们可以选择原图像中的四个点，将其应射到目标图像的四个点。然后使用opencv的 `getPerspectiveTransform` 获得变换矩阵。

### 垂直射影

In [ ]:
import cv2
import numpy as np
img = cv2.imread('/Users/davidyon/Desktop/IMG_9261.JPG')
rows, cols = img.shape[:2]
src_points = np.float32([[0,0], [cols-1,0], [0,rows-1], [cols-1,rows-1]])
dst_points = np.float32([[0,0], [cols-1,0], [int(0.33*cols),rows-1],
[int(0.66*cols),rows-1]])
projective_matrix = cv2.getPerspectiveTransform(src_points, dst_points)
img_output = cv2.warpPerspective(img, projective_matrix, (cols,rows))
cv2.imshow('Input', img)
cv2.imshow('Output', img_output)
cv2.waitKey()
cv2.destroyAllWindows()
cv2.waitKey(1)

#### 水平射影

In [ ]:
import cv2
import numpy as np
img = cv2.imread('/Users/davidyon/Desktop/IMG_9261.JPG')
rows, cols = img.shape[:2]
src_points = np.float32([[0,0], [0,rows-1], [cols/2,0],[cols/2,rows-1]])
dst_points = np.float32([[0,rows/4], [0,rows-101],
[cols/2,0],[cols/2,rows-1]])
projective_matrix = cv2.getPerspectiveTransform(src_points, dst_points)
img_output = cv2.warpPerspective(img, projective_matrix, (cols,rows))
cv2.imshow('Input', img)
cv2.imshow('Output', img_output)
cv2.waitKey()
cv2.destroyAllWindows()
cv2.waitKey(1)

## 变形（warping）

射影变换可以进行不同射影空间的转换，具有一些灵活性，但是仍然会有一些限制（一些属性保持不变）。我们可以直接对图像坐标进行操作，从而进行任何变换，我们可以通过下面几个例子学习。

In [ ]:
import cv2
import numpy as np
import math
img = cv2.imread('/Users/davidyon/Desktop/IMG_9261.JPG', cv2.IMREAD_GRAYSCALE)
rows, cols = img.shape
#####################
# Vertical wave
img_output = np.zeros(img.shape, dtype=img.dtype)
for i in range(rows):
    for j in range(cols):
        offset_x = int(50.0 * math.sin(2 * 3.14 * i / 180))
        offset_y = 0
        if j+offset_x < rows:
            img_output[i,j] = img[i,(j+offset_x)%cols]
        else:
            img_output[i,j] = 0
cv2.imshow('Input', img)
cv2.imshow('Vertical wave', img_output)
#####################
# Horizontal wave
img_output = np.zeros(img.shape, dtype=img.dtype)
for i in range(rows):
    for j in range(cols):
        offset_x = 0
        offset_y = int(32.0 * math.sin(2 * 3.14 * j / 150))
        if i+offset_y < rows:
            img_output[i,j] = img[(i+offset_y)%rows,j]
        else:
            img_output[i,j] = 0
cv2.imshow('Horizontal wave', img_output)
#####################
# Both horizontal and vertical
img_output = np.zeros(img.shape, dtype=img.dtype)
for i in range(rows):
    for j in range(cols):
        offset_x = int(40.0 * math.sin(2 * 3.14 * i / 150))
        offset_y = int(40.0 * math.cos(2 * 3.14 * j / 150))
        if i+offset_y < rows and j+offset_x < cols:
            img_output[i,j] = img[(i+offset_y)%rows,(j+offset_x)%cols]
        else:
            img_output[i,j] = 0
cv2.imshow('Multidirectional wave', img_output)
#####################
# Concave effect
img_output = np.zeros(img.shape, dtype=img.dtype)
for i in range(rows):
   for j in range(cols):
       offset_x = int(640.0 * math.sin(2 * 3.14 * i / (2*cols)))
       offset_y = 0
       if j+offset_x < cols:
           img_output[i,j] = img[i,(j+offset_x)%cols]
       else:
           img_output[i,j] = 0
cv2.imshow('Concave', img_output)
cv2.waitKey()
cv2.destroyAllWindows()
cv2.waitKey(1)

**TBD：需要仔细理解波浪化中使用变换函数**

# 边缘检测和滤波应用

## 卷积（convolution）
卷积是图像处理中的一个基本操作。它将一个数学操作应用于图像中的每个像素从而得到一个新值。这个操作是通过一个称为**核（kernel）**的矩阵实施的。对于图像中的每个像素，将核的中心点与像素点重合，然后用核矩阵中的每个值与与这个值对象的像素值进行相乘，然后将这些值加起来得到的新值就是该像素在输出图像中的值。
如果想使输出的图像与原来的图像相同，可以使用如下的等同核
$$
I = \begin{bmatrix}
     0 & 0 & 0 \\
     0 & 1 & 1 \\
     0 & 0 & 0 
     \end{bmatrix}
$$

## 虚化（Blur）
虚化（Blurring）是指将一个像素的值改成核覆盖像素的平均值。它也称为 **低通过滤波（low pass filter）** 

In [2]:
import cv2
import numpy as np
img = cv2.imread('/Users/davidyon/Desktop/IMG_9094.png')
rows, cols = img.shape[:2]
kernel_identity = np.array([[0,0,0], [0,1,0], [0,0,0]])
kernel_3x3 = np.ones((3,3), np.float32) / 9.0 # Divide by 9 to normalize the kernel
kernel_5x5 = np.ones((5,5), np.float32) / 25.0 # Divide by 25 to normalize the kernel
kernel_10x10 = np.ones((10,10), np.float32) / 100.0 # Divide by 25 to normalize the kernel
cv2.imshow('Original', img)
# value -1 is to maintain source image depth
output = cv2.filter2D(img, -1, kernel_identity)
cv2.imshow('Identity filter', output)
output = cv2.filter2D(img, -1, kernel_3x3)
cv2.imshow('3x3 filter', output)
output = cv2.filter2D(img, -1, kernel_10x10)
cv2.imshow('10x10 filter', output)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)

-1

上面的例子分别使用了等同，3x3，5x5，10x10核矩阵, 然后应用这个核矩阵到输入的图像。 `np.ones((3,3), np.float32)/9.0` 是如下的矩阵。

$$
L = {{\frac 19}} \begin{bmatrix}
        1 & 1 & 1 \\
        1 & 1 & 1 \\
        1 & 1 & 1
        \end{bmatrix}
$$

通常矩阵越大，涉及的像素越多，就会越平滑。另外除以行列的乘积，是为了用平均值作为该像素的值，从而不会人为增加该点的饱和度。

OpenCV提供了一个函数 `blur` ，使用这个函数可以将上面的代码简化为下面的一行代码。这行代码将3x3核应用到原始图像，直接产生输出。

```python
output = cv2.blur(img, (3,3))
```



## 运动虚化（Motion blur）

应用运动虚化可以产生在运动中进行拍照的效果。运动虚化核在某个特定的方向上做平均运算。它是一个有方向低通过滤波。一个3x3水平方向的运动虚化核如下：

$$
M=\begin{bmatrix}
    0 & 0 & 0 \\
    1 & 1 & 1 \\
    0 & 0 & 0
   \end{bmatrix}
$$

下面的代码给出一个运动虚化的例子

In [3]:
import cv2
import numpy as np
img = cv2.imread('/Users/davidyon/Desktop/IMG_9261.JPG')
cv2.imshow('Original', img)
size = 175
# generating the kernel
kernel_motion_blur = np.zeros((size, size))
kernel_motion_blur[int((size-1)/2), :] = np.ones(size)
kernel_motion_blur = kernel_motion_blur / size
# applying the kernel to the input image
output = cv2.filter2D(img, -1, kernel_motion_blur)
cv2.imshow('Motion Blur', output)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)

-1


我们可以设置任何方向，这个方向的像素值将被平均。同样，如果想虚化的效果更好，加大矩阵的大小。对于上面高分辨率的图，我们使用了 175x175 的核。

## 锐化（Sharpening）

锐化滤波可以使图像中对象的边缘更加锐利。这个滤波对于物体的边缘不是很清晰的图像特别有用。执行下面的代码可以看到锐化滤波的效果。

In [6]:
import cv2
import numpy as np
img = cv2.imread('/Users/davidyon/Desktop/IMG_9011.JPG')
cv2.imshow('Original', img)
# generating the kernels
kernel_sharpen_1 = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]])
kernel_sharpen_2 = np.array([[1,1,1], [1,-7,1], [1,1,1]])
kernel_sharpen_3 = np.array([[-1,-1,-1,-1,-1],
                            [-1,2,2,2,-1],
                            [-1,2,8,2,-1],
                            [-1,2,2,2,-1],
                            [-1,-1,-1,-1,-1]]) / 8.0
# applying different kernels to the input image
output_1 = cv2.filter2D(img, -1, kernel_sharpen_1)
output_2 = cv2.filter2D(img, -1, kernel_sharpen_2)
output_3 = cv2.filter2D(img, -1, kernel_sharpen_3)
cv2.imwrite('/Users/davidyon/Desktop/sharp1.jpg',output_1)
cv2.imwrite('/Users/davidyon/Desktop/sharp2.jpg',output_2)
cv2.imwrite('/Users/davidyon/Desktop/sharp3.jpg',output_3)
cv2.imshow('Sharpening', output_1)
cv2.imshow('Excessive Sharpening', output_2)
cv2.imshow('Edge Enhancement', output_3)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)

-1

锐化的程度取决于我们选择什么样的核。我们在选择核上有很大自由，每个核会产生不同的锐化效果。如果只是想正常的锐化，我们可以使用上面代码的第一种核矩阵。这个矩阵如下

$$
M = \begin{bmatrix}
    -1 & -1 & -1 \\
    -1 &  9 & -1 \\
    -1 & -1 & -1
    \end{bmatrix}
$$

如果我们想过度锐化，可以使用第二种矩阵，

$$
M = \begin{bmatrix}
    1 &  1 & 1 \\
    1 & -7 & 1 \\
    1 &  1 & 1
    \end{bmatrix}
$$

上面两个核的问题是产生的图像人工的痕迹比较明显。如果我们希望图像更自然，我们可以使用边缘增强的滤波。我们可以使用类似于高斯核（Gaussian kernel）的矩阵生成这个矩阵，这就是上面的第三种。它可以在锐化的同时进行平滑处理，从而使图像更自然。
$$
M = \frac{1}{8} \, \begin{bmatrix}
    -1 & -1 & -1 & -1 & -1 \\
    -1 &  2 &  2 &  2 & -1 \\
    -1 &  2 &  8 &  2 & -1 \\
    -1 &  2 &  2 &  2 & -1 \\
    -1 & -1 & -1 & -1 & -1 
    \end{bmatrix}
$$

你可能已经注意到了，对于前两个核，我们没有除任何数，这是应为这个核内数字的和已经等于1.

## 凸印 （Embossing）



In [6]:
import cv2
import numpy as np
img_emboss_input = cv2.imread('/Users/davidyon/Desktop/IMG_5027.jpeg')
# generating the kernels
kernel_emboss_1 = np.array([[0,-1,-1],[1,0,-1],[1,1,0]])
kernel_emboss_2 = np.array([[-1,-1,0],[-1,0,1],[0,1,1]])
kernel_emboss_3 = np.array([[1,0,0],[0,0,0],[0,0,-1]])
# converting the image to grayscale
gray_img = cv2.cvtColor(img_emboss_input,cv2.COLOR_BGR2GRAY)
# applying the kernels to the grayscale image and adding the offset to produce the shadow
output_1 = cv2.filter2D(gray_img, -1, kernel_emboss_1) + 128
output_2 = cv2.filter2D(gray_img, -1, kernel_emboss_2) + 128
output_3 = cv2.filter2D(gray_img, -1, kernel_emboss_3) + 128
cv2.imshow('Input', img_emboss_input + 128)
cv2.imshow('Embossing - South West', output_1)
cv2.imshow('Embossing - South East', output_2)
cv2.imshow('Embossing - North West', output_3)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)

-1

上面的代码使用到了下面三组核

$$
K1 = \begin{bmatrix}
     0 & -1 & -1 \\
     1 & 0  & -1 \\
     1 & 1  & 0
     \end{bmatrix}
\qquad K2 = \begin{bmatrix}
    -1 & -1 & 0 \\
    -1 &  0 & 1 \\
     0 &  1 & 1
     \end{bmatrix}
\qquad K3 = \begin{bmatrix}
     1 & 0 & 0 \\
     0 & 0 & 0 \\
     0 & 0 & -1
     \end{bmatrix}
$$

这三个核的特点是当前像素的值替换成特定方向的相邻像素的差值。凸印的效果是由将图像中所有像素的值偏移128得到，这个操作在图片中增加了高亮/阴影的效果。

## 边缘检测（edge detection）
边缘检测是图像处理核计算机视觉中的基本问题，边缘检测的目的是标识数字图像中亮度变化明显的点。图像属性中的明显变化通常反映了属性的重要事件和变化。包括：
* 深度的不连续
* 表面方向不连续
* 物质属性变化
* 场景照明变化

边缘是对象和背景之间的边界，还能表示重叠对象之间的边界。从图像的角度来看，是指周围像素灰度急剧变化的那些像素的集合。边缘检测是图像分割的一部分，图像分割的目的是识别出图像中的区域。边缘检测是定位边缘像素的过程，而边缘增强是增加边缘和背景之间的对比度以便能够更清楚地看清边缘的过程。边缘跟踪是沿着边缘进行跟踪的过程，这个过程通常会把边缘像素采集到一个列表中，链码算法是边缘跟踪算法的一个特例。

边缘检测有多种方法，但是它们的绝大部分可以划归为两类：基于查找（based-search）和基于零穿越（zero-crossing）。基于查找的方法通过寻找图像一阶导数中的最大和最小值来检测边界，通常将边界定位在梯度最大的方向。基于零穿越的方法通过寻找图像二阶导数零穿越来寻找边界，通常是Laplacian过零点或者非线性差分表示的过零点。



### 一阶导数
许多边缘检测操作是基于亮度的一阶导数 -- 这样就得到了原始数据亮度的梯度。使用这个信息我们能够在图像的亮度梯度中寻找峰值。

$
{\displaystyle 
    {\begin{aligned}
    L_{x}(x,y) =-{\frac {1}{2}}L(x-1,y)+0\cdot L(x,y)+{\frac {1}{2}}\cdot L(x+1,y) \\
    L_{y}(x,y) =-{\frac {1}{2}}L(x,y-1)+0\cdot L(x,y)+{\frac {1}{2}}\cdot L(x,y+1)
    \end{aligned}}}
$

其中 $L_{x}(x,y)$ 是水平方向的亮度梯度， $L_{y}(x,y)$ 是垂直方向的亮度梯度。相应的应用于图像数据的带有掩码的核如下

$
{
\displaystyle 
L_{x}={\begin{bmatrix} + ^1\!/_2 & 0 & - ^1\!/_2 \end{bmatrix}}L\quad 
{\text{and}}\quad 
L_{y}={\begin{bmatrix}+ ^1\!/_2\\0\\- ^1\!/_2\end{bmatrix}}L.}
$

著名的 **Sobel** 边缘检测算子基于如下的核滤波：

$
{\displaystyle
    L_x = \begin{bmatrix} +1 & 0 & -1 \\ +2 & 0 & -2 \\ +1 & 0 & -1 \end{bmatrix}L \quad \text{and} \quad
    L_y = \begin{bmatrix} +1 & +2 & +1 \\ 0 & 0 & 0 \\ -1 & -2 & -1 \end{bmatrix}L.
}
$

有了上面一阶图像导数，我们可以得到梯度值（gradient magnitude）为：

$ |\Delta L| = \sqrt{L_x^2 + L_y^2} $

方向为

$ \theta = \arctan \big( \frac{L_y}{L_x} \big) $

In [12]:
import cv2
import numpy as np
img = cv2.imread('image/shape.jpg', cv2.IMREAD_GRAYSCALE)
img = cv2.imread('image/town.jpg', cv2.IMREAD_GRAYSCALE)
rows, cols = img.shape
# It is used depth of cv2.CV_64F.
sobel_horizontal = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=3)
# Kernel size can be: 1,3,5 or 7.
sobel_vertical = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=3)
laplacian = cv2.Laplacian(img,cv2.CV_64F)
canny = cv2.Canny(img, 10,240)
cv2.imshow('Original', img)
cv2.imshow('Sobel horizontal', sobel_horizontal)
cv2.imshow('Sobel vertical', sobel_vertical)
cv2.imshow('Laplacian', laplacian)
cv2.imshow('Canny', canny)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)

-1

为了获得希望的结果，核的大小-也就是ksize，值越小，边越细。正如输出所示，Sobel滤波只能检测水平方向或者垂直方向的边，为了克服这个问题，我们使用 Laplacian 滤波，它能够检测两个方向的边。尽管 Laplacian 在简单形状的检测中表现还可以，如例子中的shape.jpg。
![Shape](image/shape.jpg)
但是对于复杂的图形就很难达到预期的效果。这时候可以选择经常使用的Canny算法。
![古镇](image/town.jpg)

对于上图，Laplacian核产生了很多躁点，而Canny的边检测却好的多。它带有两个数值参数分别作为低门限值和高门限值。如果梯度值大于高门限值，它将会标记为强边，如果梯度值介于两个门限值之间，则与高门限值想连的点被选择作为强边，这个过程持续到低门限值。低于低门限值的点将被忽略。这样获得的输出会更清晰和锐利。